___

<a href='https://github.com/eliasmelul/'> <img src='https://s3.us-east-2.amazonaws.com/wordontheamazon.com/NoMargin_NewLogo.png' style='width: 15em;' align='right' /></a>
# Finding my Schitt's Creek
#### Data Collection: Weather and City Names
___
<h3 align="right">by Elias Melul, Data Scientist </h3> 

___



## Collecting Weather Data and City Names

---
**GOOD NEWS!** If you want to download this data, you don't need to rerun and scrap the web. I've made the data available for you **<a href='https://github.com/eliasmelul/finding_schitts/blob/master/Data/final_weather_data.csv'>here</a>**.

Want direct access? **<a href='https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/final_weather_data.csv'>Click Here</a>**

---
This data includes the average monthly temperature (high and low), the average monthly precipitation and the average monthly snowfall of about 5800 cities.

----


<h2 id = "DataCollection">Data Collection</h2>

In order to create a competent recommender system, I need information from cities in the USA. We will then analyze such data, select the features we want to use, and begin building!

----
**Which cities will I include?**

The cities to be included are those that are listed in <a href='https://www.usclimatedata.com/climate/united-states/us'>this</a> website. 


**_States:_** 

<a><img src='https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Images/USA_States_Table_WSite.PNG' style='width: 30em;' align=auto /></a>

As you can observe in the image, the website contains a table that references all the states in the US. 

<a><img src='https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Images/States_Cities_Table_WSite.PNG' style='width: 30em;' align=auto /></a>

Once you click on one of the states, like Alabama in this example, a table of all the cities in such state appears. 

Therefore, we will user this website and BeautifulSoup to get all the cities in each state, and the references (URLs) to each of the cities in each state so we can loop through these and scrap all the weather data.


----


#### Import Libraries

---
The libraries imported are not all used in this notebook. However, to be able to follow the whole project, please make sure you have these installed!

Otherwise use pip or conda to install them on your laptop or computer.

In [18]:
import pandas as pd # import pandas for dataframes
import numpy as np
import requests
from bs4 import BeautifulSoup
import locale
from datetime import datetime
import re
import matplotlib.pyplot as plt

from IPython.display import HTML, display
from IPython.display import Image 
from IPython.core.display import HTML 

pd.options.display.max_columns = None
pd.options.display.max_rows=None


%matplotlib inline

<h3 id ="GettingList">Getting List of Cities</h3>

---

To get the list of cities, we will be be using <a href='https://www.usclimatedata.com/climate/united-states/us'>this</a> weather website. This will allow is to get the list of states and the names of each city in every state along with the weather data for each city in each state.
1. Get the name of each state along with the link to the websites that contains all the cities in each state. 
2. Get the name of each city along with the link to the websites that contains all the weather information in each city.
3. Scrap the desired weather data for each city in every state as specified in this website.
    - Average monthly temperature
    - Swing from average high to average low temperature for every month.

#### Generating the URLs

In [2]:
# Create list of tuples containing the name of the state and the href
def get_state_names(url):
    states = []
    req = requests.get(url)
    soup = BeautifulSoup(req.content, features='lxml')
    table2 = soup.find_all("a",{"class":"stretched-link"})
    for j in range(0,len(table2)):
        newurl = 'https://www.usclimatedata.com'+table2[j].get('href')
        name_state = table2[j].get_text()
        tempdic = {'State':name_state,'url':newurl}
        states.append(tempdic)
    states_df = pd.DataFrame.from_dict(states)
    return states_df
states_names_list = get_state_names('https://www.usclimatedata.com/')

District of Columbia is weirdly defined in our dataframe... lets fix it.

In [3]:
states_names_list.at[8,'State'] = "District of Columbia"
states_names_list[0:5]

,State,url
0,Alabama,https://www.usclimatedata.com/climate/alabama/...
1,Alaska,https://www.usclimatedata.com/climate/alaska/u...
2,Arizona,https://www.usclimatedata.com/climate/arizona/...
3,Arkansas,https://www.usclimatedata.com/climate/arkansas...
4,California,https://www.usclimatedata.com/climate/californ...


In [4]:
def get_city_names(url):
    cities_st = []
    req = requests.get(url)
    soup = BeautifulSoup(req.content, features='lxml')
    name_state = soup.find("p",{"class":"selection_title"})
    name_state = name_state.get_text()[:-24]
    table1 = soup.find_all("a", {"class":"stretched-link"})
    for i in range(0,len(table1)):
        name_city = table1[i].get_text()
        newurl = 'https://www.usclimatedata.com'+table1[i].get('href')
        tempdic = {'State':name_state,'City':name_city,'url_city':newurl}
        cities_st.append(tempdic)
    cities_df = pd.DataFrame(cities_st)
    return cities_df
excities = get_city_names('https://www.usclimatedata.com/climate/alabama/united-states/3170')
excities.head()

,State,City,url_city
0,Alabama,Addison,https://www.usclimatedata.com/climate/addison/...
1,Alabama,Alabaster,https://www.usclimatedata.com/climate/alabaste...
2,Alabama,Alexander City,https://www.usclimatedata.com/climate/alexande...
3,Alabama,Aliceville,https://www.usclimatedata.com/climate/alicevil...
4,Alabama,Andalusia,https://www.usclimatedata.com/climate/andalusi...


In [5]:
state_city_urls = []
for i, row in states_names_list.iterrows():
    random_state_city = get_city_names(row.url)
    state_city_urls.append(random_state_city)


In [6]:
state_city_df=pd.DataFrame()
for i in range(0,len(state_city_urls)):
    ith_df = state_city_urls[i]
    state_city_df = state_city_df.append(ith_df, ignore_index=True)
state_city_df.head()

,State,City,url_city
0,Alabama,Addison,https://www.usclimatedata.com/climate/addison/...
1,Alabama,Alabaster,https://www.usclimatedata.com/climate/alabaste...
2,Alabama,Alexander City,https://www.usclimatedata.com/climate/alexande...
3,Alabama,Aliceville,https://www.usclimatedata.com/climate/alicevil...
4,Alabama,Andalusia,https://www.usclimatedata.com/climate/andalusi...


In [7]:
print(state_city_df.shape)
state_city_df.head()

(5850, 3)


,State,City,url_city
0,Alabama,Addison,https://www.usclimatedata.com/climate/addison/...
1,Alabama,Alabaster,https://www.usclimatedata.com/climate/alabaste...
2,Alabama,Alexander City,https://www.usclimatedata.com/climate/alexande...
3,Alabama,Aliceville,https://www.usclimatedata.com/climate/alicevil...
4,Alabama,Andalusia,https://www.usclimatedata.com/climate/andalusi...


---

Now that we have a dataframe with all the cities and their respective URL to scrap weather data, lets begin scraping!

In [8]:
#Now, let's get the average highs, lows and precipitation in each city
# def get_weather(state, city, url):
#     req = requests.get(url)
#     soup = BeautifulSoup(req.content, features='lxml')
#     temp_high = soup.find_all("td",{"class":"high text-right"})
#     temp_high = [i.get_text() for i in temp_high]
#     temp_high = temp_high[0:12]
#     temp_low = soup.find_all("td",{"class":"low text-right"})
#     temp_low = [i.get_text() for i in temp_low]
#     temp_low = temp_low[0:12]
#     precip = soup.find_all("td",{"class":"text-right"})
#     precip = [i.get_text() for i in precip]
#     precip = precip[12:18]+precip[30:36]
    
#     swing = [float(temp_high[i])-float(temp_low[i]) for i in range(0,len(temp_high))]
#     avg_day = [(float(temp_high[i])+float(temp_low[i]))/2 for i in range(0,len(temp_high))]
#     ava = avg_day+swing+precip
#     df = pd.DataFrame(ava, index = ['Temp Jan','Temp Feb','Temp Mar','Temp Apr','Temp May','Temp Jun','Temp Jul','Temp Aug','Temp Sep','Temp Oct','Temp Nov','Temp Dec','Swing Jan','Swing Feb','Swing Mar','Swing Apr','Swing May','Swing Jun','Swing Jul','Swing Aug','Swing Sep','Swing Oct','Swing Nov','Swing Dec','Precip Jan','Precip Feb','Precip Mar','Precip Apr','Precip May','Precip Jun','Precip Jul','Precip Aug','Precip Sep','Precip Oct','Precip Nov','Precip Dec'])
#     df = df.transpose()
#     df['State'] = state
#     df['City'] = city
#     return precip
# get_weather('Florida','Apalachicola','https://www.usclimatedata.com/climate/atlanta/georgia/united-states/usga1761')

**Scrap weather parameters: NEW VERSION - pandas**

In [19]:
def get_weather(state, city, url):
    req = pd.read_html(url)
    janjun = req[0]
    juldec = req[1]
    
    for i, row in janjun.iterrows():
        if ("Average high" in row['Unnamed: 0']):
            high = [i for i in row[1:]]
        if ("Average low" in row['Unnamed: 0']):
            low = [i for i in row[1:]]
        if ("precipitation in inch" in row['Unnamed: 0']):
            prec = [i for i in row[1:]]
        else:
            prec = [0,0,0,0,0,0]
        if ("snowfall in inch" in row['Unnamed: 0']):
            snow = [i for i in row[1:]]
        else:
            snow = [0,0,0,0,0,0]
            
    for i, row in juldec.iterrows():
        if ("Average high" in row['Unnamed: 0']):
            high2 = [i for i in row[1:]]
        if ("Average low" in row['Unnamed: 0']):
            low2 = [i for i in row[1:]]
        if ("precipitation in inch" in row['Unnamed: 0']):
            prec2 = [i for i in row[1:]]
        else:
            prec2 = [0,0,0,0,0,0]
        if ("snowfall in inch" in row['Unnamed: 0']):
            snow2 = [i for i in row[1:]]
        else:
            snow2 = [0,0,0,0,0,0]

    highs = high+high2
    lows = low+low2
    precip = prec+prec2
    snows = snow+snow2
    
    highD = {"Jan High":highs[0],"Feb High":highs[1],"Mar High":highs[2],"Apr High":highs[3],"May High":highs[4],"Jun High":highs[5],"Jul High":highs[6],"Aug High":highs[7],"Sep High":highs[8],"Oct High":highs[9],"Nov High":highs[10],"Dec High":highs[11]}
    lowD = {"Jan Low":lows[0],"Feb Low":lows[1],"Mar Low":lows[2],"Apr Low":lows[3],"May Low":lows[4],"Jun Low":lows[5],"Jul Low":lows[6],"Aug Low":lows[7],"Sep Low":lows[8],"Oct Low":lows[9],"Nov Low":lows[10],"Dec Low":lows[11]}
    precipD = {"Jan Prec":precip[0],"Feb Prec":precip[1],"Mar Prec":precip[2],"Apr Prec":precip[3],"May Prec":precip[4],"Jun Prec":precip[5],"Jul Prec":precip[6],"Aug Prec":precip[7],"Sep Prec":precip[8],"Oct Prec":precip[9],"Nov Prec":precip[10],"Dec Prec":precip[11]}
    snowD = {"Jan Snow":snows[0],"Feb Snow":snows[1],"Mar Snow":snows[2],"Apr Snow":snows[3],"May Snow":snows[4],"Jun Snow":snows[5],"Jul Snow":snows[6],"Aug Snow":snows[7],"Sep Snow":snows[8],"Oct Snow":snows[9],"Nov Snow":snows[10],"Dec Snow":snows[11]}
    
    #
    combined=pd.DataFrame.from_dict({**highD,**lowD, **precipD, **snowD}, orient="index")
    combined = combined.transpose()
    combined['State'] = state
    combined['City'] = city
    return combined
get_weather('New York','Rochester','https://www.usclimatedata.com/climate/rochester/new-york/united-states/usny1232')

,Jan High,Feb High,Mar High,Apr High,May High,Jun High,Jul High,Aug High,Sep High,Oct High,Nov High,Dec High,Jan Low,Feb Low,Mar Low,Apr Low,May Low,Jun Low,Jul Low,Aug Low,Sep Low,Oct Low,Nov Low,Dec Low,Jan Prec,Feb Prec,Mar Prec,Apr Prec,May Prec,Jun Prec,Jul Prec,Aug Prec,Sep Prec,Oct Prec,Nov Prec,Dec Prec,Jan Snow,Feb Snow,Mar Snow,Apr Snow,May Snow,Jun Snow,Jul Snow,Aug Snow,Sep Snow,Oct Snow,Nov Snow,Dec Snow,State,City
0,32.0,34.0,43.0,56.0,68.0,77.0,81.0,79.0,72.0,60.0,48.0,37.0,18.0,19.0,26.0,37.0,46.0,56.0,61.0,60.0,52.0,42.0,33.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,22.0,16.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,22.0,New York,Rochester


**The next step takes a while, depending on your internet connection! Expect it to run somewhere around 30min!**

In [10]:
all_weather_df = pd.DataFrame()
for i, row in state_city_df.iterrows():
    try:
        newweather = get_weather(row.State, row.City, row.url_city)
        all_weather_df = all_weather_df.append(newweather, ignore_index=True)
    except:
        pass

In [11]:
print(all_weather_df.shape)
all_weather_df.head()

(5809, 50)


,Jan High,Feb High,Mar High,Apr High,May High,Jun High,Jul High,Aug High,Sep High,Oct High,Nov High,Dec High,Jan Low,Feb Low,Mar Low,Apr Low,May Low,Jun Low,Jul Low,Aug Low,Sep Low,Oct Low,Nov Low,Dec Low,Jan Prec,Feb Prec,Mar Prec,Apr Prec,May Prec,Jun Prec,Jul Prec,Aug Prec,Sep Prec,Oct Prec,Nov Prec,Dec Prec,Jan Snow,Feb Snow,Mar Snow,Apr Snow,May Snow,Jun Snow,Jul Snow,Aug Snow,Sep Snow,Oct Snow,Nov Snow,Dec Snow,State,City
0,50.0,54.0,63.0,71.0,79.0,85.0,89.0,89.0,83.0,73.0,63.0,52.0,29.0,33.0,40.0,48.0,56.0,64.0,68.0,67.0,60.0,48.0,40.0,32.0,5.03,5.29,5.25,4.78,5.48,4.83,4.83,3.75,4.27,3.77,5.05,5.91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alabama,Addison
1,54.0,59.0,67.0,74.0,81.0,87.0,91.0,90.0,85.0,75.0,66.0,56.0,34.0,38.0,44.0,51.0,60.0,68.0,71.0,70.0,64.0,53.0,44.0,37.0,5.33,5.75,4.48,3.89,4.48,4.63,4.76,4.76,3.83,3.02,4.95,4.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alabama,Alabaster
2,55.0,59.0,68.0,75.0,82.0,88.0,91.0,90.0,85.0,76.0,67.0,57.0,32.0,35.0,41.0,48.0,57.0,66.0,69.0,68.0,62.0,50.0,41.0,34.0,5.21,5.35,5.49,4.11,4.33,4.45,5.31,4.50,4.10,3.08,4.79,4.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alabama,Alexander City
3,56.0,60.0,69.0,77.0,84.0,90.0,92.0,92.0,87.0,78.0,68.0,58.0,31.0,34.0,42.0,49.0,57.0,66.0,70.0,69.0,62.0,50.0,41.0,33.0,5.24,5.59,5.27,4.86,4.60,4.88,4.99,3.57,3.76,3.97,4.84,4.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alabama,Aliceville
4,61.0,65.0,72.0,78.0,85.0,90.0,92.0,91.0,88.0,80.0,72.0,63.0,32.0,35.0,41.0,47.0,56.0,65.0,67.0,68.0,62.0,50.0,40.0,35.0,5.18,5.30,6.35,4.17,3.89,5.30,6.16,5.65,4.62,3.71,4.73,4.91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alabama,Andalusia


**State abbreviations**

We are going to add the abbreviation to each state for future analytical purposes. These will become clear in the Data Collection - DATAUSA section.

---
To add the abbreviations, we must get a list of abbreviations and their respective states spelled out. Luckily, Wikipedia has a site listing all the US abbreviations! We will first import them, and then add the appropiate abbreviation to each City in our dataframe.

In [12]:
abb_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations', skiprows=11)[0]
abb_df = abb_df[['United States of America', 'Unnamed: 5']]
abb_df = abb_df.rename(columns={'United States of America':'State','Unnamed: 5':'State_Abb'})

In [13]:
#Now let's create a list of urls for further scrapping
weatherCity_df = all_weather_df
weatherCity_df = weatherCity_df.merge(abb_df, on='State')
weatherCity_df['CityState'] = [row.City+", "+row.State_Abb for i, row in weatherCity_df.iterrows()]

#Shift city and state column to firsts
cols = weatherCity_df.columns.tolist()
cols = cols[-4:]+cols[:-4]
weatherCity_df = weatherCity_df[cols]


In [14]:
weatherCity_df.head()

,State,City,State_Abb,CityState,Jan High,Feb High,Mar High,Apr High,May High,Jun High,Jul High,Aug High,Sep High,Oct High,Nov High,Dec High,Jan Low,Feb Low,Mar Low,Apr Low,May Low,Jun Low,Jul Low,Aug Low,Sep Low,Oct Low,Nov Low,Dec Low,Jan Prec,Feb Prec,Mar Prec,Apr Prec,May Prec,Jun Prec,Jul Prec,Aug Prec,Sep Prec,Oct Prec,Nov Prec,Dec Prec,Jan Snow,Feb Snow,Mar Snow,Apr Snow,May Snow,Jun Snow,Jul Snow,Aug Snow,Sep Snow,Oct Snow,Nov Snow,Dec Snow
0,Alabama,Addison,AL,"Addison, AL",50.0,54.0,63.0,71.0,79.0,85.0,89.0,89.0,83.0,73.0,63.0,52.0,29.0,33.0,40.0,48.0,56.0,64.0,68.0,67.0,60.0,48.0,40.0,32.0,5.03,5.29,5.25,4.78,5.48,4.83,4.83,3.75,4.27,3.77,5.05,5.91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alabama,Alabaster,AL,"Alabaster, AL",54.0,59.0,67.0,74.0,81.0,87.0,91.0,90.0,85.0,75.0,66.0,56.0,34.0,38.0,44.0,51.0,60.0,68.0,71.0,70.0,64.0,53.0,44.0,37.0,5.33,5.75,4.48,3.89,4.48,4.63,4.76,4.76,3.83,3.02,4.95,4.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alabama,Alexander City,AL,"Alexander City, AL",55.0,59.0,68.0,75.0,82.0,88.0,91.0,90.0,85.0,76.0,67.0,57.0,32.0,35.0,41.0,48.0,57.0,66.0,69.0,68.0,62.0,50.0,41.0,34.0,5.21,5.35,5.49,4.11,4.33,4.45,5.31,4.50,4.10,3.08,4.79,4.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alabama,Aliceville,AL,"Aliceville, AL",56.0,60.0,69.0,77.0,84.0,90.0,92.0,92.0,87.0,78.0,68.0,58.0,31.0,34.0,42.0,49.0,57.0,66.0,70.0,69.0,62.0,50.0,41.0,33.0,5.24,5.59,5.27,4.86,4.60,4.88,4.99,3.57,3.76,3.97,4.84,4.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alabama,Andalusia,AL,"Andalusia, AL",61.0,65.0,72.0,78.0,85.0,90.0,92.0,91.0,88.0,80.0,72.0,63.0,32.0,35.0,41.0,47.0,56.0,65.0,67.0,68.0,62.0,50.0,40.0,35.0,5.18,5.30,6.35,4.17,3.89,5.30,6.16,5.65,4.62,3.71,4.73,4.91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


There it goes! We've scraped the weather information and structured it in our desired format. Now, lets go to the next section to scrap the general information for each city.

In [16]:
weatherCity_df.shape

(5963, 52)

In [17]:
weatherCity_df[weatherCity_df.State=='New York']

,State,City,State_Abb,CityState,Jan High,Feb High,Mar High,Apr High,May High,Jun High,Jul High,Aug High,Sep High,Oct High,Nov High,Dec High,Jan Low,Feb Low,Mar Low,Apr Low,May Low,Jun Low,Jul Low,Aug Low,Sep Low,Oct Low,Nov Low,Dec Low,Jan Prec,Feb Prec,Mar Prec,Apr Prec,May Prec,Jun Prec,Jul Prec,Aug Prec,Sep Prec,Oct Prec,Nov Prec,Dec Prec,Jan Snow,Feb Snow,Mar Snow,Apr Snow,May Snow,Jun Snow,Jul Snow,Aug Snow,Sep Snow,Oct Snow,Nov Snow,Dec Snow
3491,New York,Addison,NY,"Addison, NY",33.0,36.0,44.0,58.0,69.0,77.0,81.0,79.0,72.0,60.0,49.0,36.0,14.0,16.0,22.0,32.0,41.0,51.0,56.0,55.0,48.0,37.0,29.0,20.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.0,11.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,9.0
3492,New York,Albany,NY,"Albany, NY",31.0,35.0,44.0,58.0,69.0,78.0,82.0,80.0,72.0,60.0,48.0,36.0,15.0,17.0,26.0,37.0,47.0,56.0,61.0,60.0,52.0,40.0,31.0,21.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.0,12.0,10.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,14.0
3493,New York,Albion,NY,"Albion, NY",32.0,35.0,44.0,57.0,69.0,78.0,82.0,80.0,73.0,60.0,48.0,37.0,18.0,19.0,26.0,37.0,47.0,57.0,62.0,60.0,54.0,43.0,34.0,24.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20.0,14.0,12.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,14.0
3494,New York,Alcove,NY,"Alcove, NY",31.0,34.0,43.0,56.0,67.0,75.0,80.0,78.0,71.0,59.0,48.0,36.0,11.0,13.0,22.0,34.0,44.0,54.0,59.0,57.0,48.0,37.0,29.0,19.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.0,11.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.0
3495,New York,Alfred,NY,"Alfred, NY",31.0,35.0,44.0,55.0,68.0,76.0,80.0,79.0,71.0,60.0,47.0,36.0,13.0,14.0,22.0,28.0,42.0,52.0,56.0,54.0,47.0,37.0,28.0,19.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20.0,17.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,18.0
3496,New York,Angelica,NY,"Angelica, NY",33.0,36.0,45.0,58.0,69.0,76.0,80.0,78.0,71.0,60.0,48.0,37.0,12.0,12.0,19.0,29.0,39.0,49.0,53.0,52.0,44.0,34.0,28.0,18.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.0,12.0,12.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,15.0
3497,New York,Arcade,NY,"Arcade, NY",31.0,33.0,43.0,56.0,68.0,76.0,80.0,78.0,71.0,59.0,46.0,35.0,14.0,15.0,23.0,33.0,42.0,51.0,56.0,54.0,48.0,38.0,31.0,20.0,2.80,2.32,2.76,3.27,3.66,4.69,3.86,3.86,4.61,3.43,3.82,3.54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3498,New York,Ardsley,NY,"Ardsley, NY",38.0,42.0,51.0,62.0,72.0,81.0,85.0,83.0,76.0,65.0,54.0,43.0,22.0,24.0,30.0,39.0,49.0,58.0,63.0,62.0,55.0,44.0,36.0,27.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.0,10.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0
3499,New York,Auburn,NY,"Auburn, NY",31.0,34.0,43.0,56.0,69.0,78.0,82.0,80.0,72.0,60.0,48.0,36.0,15.0,15.0,23.0,34.0,44.0,55.0,60.0,58.0,50.0,40.0,31.0,22.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,33.0,23.0,15.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,25.0
3500,New York,Aurora,NY,"Aurora, NY",32.0,34.0,42.0,56.0,68.0,77.0,81.0,80.0,73.0,60.0,48.0,37.0,17.0,18.0,25.0,37.0,47.0,57.0,61.0,59.0,52.0,42.0,34.0,24.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.0,13.0,12.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,12.0
